In [67]:
import numpy as np
import pathlib
import os
import glob
import shutil
import re
import cv2
import pandas as pd
import pickle
import matplotlib.pyplot as plt

from math import ceil
from PIL import Image
from datetime import datetime

from sklearn.preprocessing import LabelBinarizer, MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.metrics import precision_score, recall_score, f1_score

import tensorflow as tf
from keras import backend as K
from keras import Input
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers import GlobalAveragePooling2D
from keras.layers.core import Activation, Flatten, Dropout, Dense
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img, array_to_img
from keras.optimizers import Adam
from keras.models import Model, load_model
from keras.losses import BinaryCrossentropy
from keras.applications.vgg16 import VGG16
from keras.applications.mobilenet import MobileNet
from keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from keras.applications.inception_v3 import InceptionV3
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, TensorBoard
from keras.metrics import Accuracy, Precision, Recall, CategoricalAccuracy

tf.compat.v1.ConfigProto().gpu_options.allow_growth = True
tf.config.run_functions_eagerly(False)

In [2]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


# Load data

In [3]:
# Directories containing input data
AUGMENTED_DATA = os.path.join('data', 'Plant_leave_diseases_dataset_with_augmentation')
RAW_DATA = os.path.join('data', 'Plant_leave_diseases_dataset_without_augmentation')
DATA_DIRECTORY = os.path.join('data', 'PlantVillage')
MODELS_DIRECTORY = os.path.join('data', 'models')
LABELS_DIRECTORY = os.path.join('data', 'models', 'labels')

LABELS = [
    "Scab",
    "Black_Rot",
    "Rust",
    "Background",
    "Healthy",
    "Powdery_Mildew",
    "Leaf_Spot",
    "Bacterial_Spot",
    "Target_Spot",
    "Mosaic_Virus",
    "Yellow_Leaf_Curl_Virus",
    "Leaf_Scorch",
    "Leaf_Mold",
    "Spider_Mites",
    "Black_Measles",
    "Citrus_Greening",
    "Blight"
]

# Dimensions of resized image
IMAGE_SIZE = (224, 224)

INPUT_SHAPE = (*IMAGE_SIZE, 3)

# Due to memory limitations a smaller portion of input images needs to be taken.
IMAGE_LIMIT = 12500

In [4]:
def transform_image(image_path: str, image_size: tuple):
    image = cv2.imread(image_path)
    resized_image = cv2.resize(image, image_size)
    transformed_image =  img_to_array(resized_image)
    return transformed_image

In [5]:
def clean_label(label: str) -> str:
    whitespace = label.find(' ')
    if whitespace > -1:
        label = f"{label[:whitespace]}_({label[whitespace + 1:].replace(' ', '_')})"
    
    floor = re.search('_{2,}', label)
    if floor:
        regex = re.compile('(_?tomato_?|_?apple_?)', re.IGNORECASE)
        label = re.sub(regex, '_', label[floor.end():])
    
    return label.title().strip('_')

In [6]:
def unpack(data):
    for value in data.values():
        if isinstance(value, dict):
            yield from unpack(value)
        else:
            if isinstance(value, list):
                yield np.asarray(value)
            else:
                yield value

In [7]:
def search_for_label(folder):
    for label in LABELS:
        if label in folder:
            return label
    return None

In [9]:
def prepare_data(output_directory: str, image_limit: int = -1, use_labels=False, use_augmented=False, shuffle=False, copy=False):
    # Original database consists of raw & augmented dataset
    datasets = {
        'raw': RAW_DATA, 
    }
    
    if use_augmented:
        datasets['augmented'] = AUGMENTED_DATA
    
    # Number of images per label
    images_per_label = image_limit // 39
    
    # Images per dataset = Number of images (and labels) / Number of datasets
    images_per_dataset = images_per_label // len(datasets) if images_per_label > -1 else -1
    
    # Make sure to provide the number of images_per_label high enough to split raw & augmented data into train and test set
    # Therefore min_images_per_label is 6: ceil(6 / 2) = 3 => round(3 * 0.8) = 2 => train, test = 2, 1
    if images_per_dataset == -1 or ceil(0.9 * images_per_dataset) < images_per_dataset:
        
        print('🛠️ Data preparation in progress...')
        
        # Dictionary storing data
        data = {
            split: {
                key: [] for key in ['data', 'labels']
            } for split in ['train', 'test']
        }
        
        # Iterate over raw and augmented datasets
        for key, dir_path in datasets.items():
            print(f'\n\t⚙️ Loading {key} data...\n')
            for root, directories, files in os.walk(dir_path):
                folder = os.path.basename(root)
                loaded = 0
                if files: 
                    label = clean_label(folder)
                    if use_labels:
                        label = search_for_label(label)
                        assert label is not None, "Correspondig label should exist!"
                    if shuffle:
                        np.random.seed(42)
                        np.random.shuffle(files)

                    if images_per_dataset == -1:
                        N = len(files)
                    else:
                        N = images_per_dataset

                    # Split data into train and test set (9:1)
                    split_idx = [0, ceil(0.9 * N), N]

                    for i, split in enumerate(data):
                        if copy:
                            new_directory = os.path.join(output_directory, split, label)
                            if not os.path.isdir(new_directory):
                                os.makedirs(new_directory)
                            else:
                                shutil.rmtree(new_directory)
                                os.makedirs(new_directory)

                        start = split_idx[i]
                        end = split_idx[i + 1]

                        for j, file in enumerate(files[start: end], start=start + 1):
                            current_path = os.path.join(root, file)
                            if copy:
                                _, extension = os.path.splitext(file)
                                filename = f'{label}_{key}_{j}{extension}'
                                image_path = os.path.join(new_directory, filename)
                                shutil.copy(current_path, image_path)
                            else:
                                image_path = current_path
                            try:
                                image = transform_image(image_path, IMAGE_SIZE)
                                data[split]['data'].append(image)
                                data[split]['labels'].append(label)
                                loaded += 1
                                # print(f'[✔️] {image_path} successfully loaded')
                            # Broad exception clause due to possible cv2 errors
                            except Exception as error: # ValueError as error:
                                print(f'\t\t[❌] Error handled for {image_path}: {error}')
            
                    print(f"\t\t✔️ [{key.upper()}] {folder} ({label}): Successfully loaded {loaded} / {images_per_dataset} images")
                    
        
        print('\n\n📊 Summary:\n')
        for split in data:
            print(f"\t🗂️ {split.capitalize()}ing dataset contains of {len(data[split]['labels'])} images")
        
        return np.array(list(unpack(data)), dtype='object')
        
    else:
        print('[❌] Empty data returned. The image limit should be higher!')
        return np.empty(shape=(4,), dtype='object')

In [10]:
# Train and test data|
train_directory = os.path.join(DATA_DIRECTORY, 'train')
test_directory = os.path.join(DATA_DIRECTORY, 'test')

# Prepare data directories
X, y, X_test, y_test = prepare_data(
    DATA_DIRECTORY, 
    image_limit=IMAGE_LIMIT, 
    use_labels=True,
    use_augmented=False, 
    shuffle=True, 
    copy=False
)

🛠️ Data preparation in progress...

	⚙️ Loading raw data...

		✔️ [RAW] Apple___Apple_scab (Scab): Successfully loaded 320 / 320 images
		✔️ [RAW] Apple___Black_rot (Black_Rot): Successfully loaded 320 / 320 images
		✔️ [RAW] Apple___Cedar_apple_rust (Rust): Successfully loaded 275 / 320 images
		✔️ [RAW] Apple___healthy (Healthy): Successfully loaded 320 / 320 images
		✔️ [RAW] Background_without_leaves (Background): Successfully loaded 320 / 320 images
		✔️ [RAW] Blueberry___healthy (Healthy): Successfully loaded 320 / 320 images
		✔️ [RAW] Cherry___healthy (Healthy): Successfully loaded 320 / 320 images
		✔️ [RAW] Cherry___Powdery_mildew (Powdery_Mildew): Successfully loaded 320 / 320 images
		✔️ [RAW] Corn___Cercospora_leaf_spot Gray_leaf_spot (Leaf_Spot): Successfully loaded 320 / 320 images
		✔️ [RAW] Corn___Common_rust (Rust): Successfully loaded 320 / 320 images
		✔️ [RAW] Corn___healthy (Healthy): Successfully loaded 320 / 320 images
		✔️ [RAW] Corn___Northern_Leaf_Blight (Bli

# Binarize training labels

In [11]:
def encode_labels(y, encoder, fit=False, save=False):
    if fit:
        y_bin = encoder.fit_transform(y)
        unique_binaries = pd.unique([np.argmax(x) for x in y_bin])
        unique_labels = pd.unique(y)

        assert dict(zip(unique_binaries, unique_labels)) == {np.argmax(x): y[i] for i, x in enumerate(y_bin)}, 'Ambiguity in labels!'

        encoded_labels = dict(zip(unique_binaries, unique_labels))
        if save:
            if not os.path.isdir(LABELS_DIRECTORY):
                os.makedirs(LABELS_DIRECTORY, exist_ok=True)
            output_pickle = os.path.join(LABELS_DIRECTORY, 'happy_plant_labels.pickle')
            with open(output_pickle, 'wb') as file:
                pickle.dump(encoded_labels, file)
                print(f'💾 Labels saved as {output_pickle}\n')
        return y_bin, encoded_labels
    else:
        y_bin = encoder.transform(y)
        return y_bin

In [12]:
label_binarizer = LabelBinarizer() 

# Binarization of multiple classes can increase the accuracy
y_bin, encoded_labels = encode_labels(y, label_binarizer, fit=True, save=True)

print('🦕 Encoded labels:\n')
for k, v in sorted(encoded_labels.items()):
    print(f'\t{k}: {v}')
    
n_classes = len(encoded_labels)
print(f'\n🪐 There are {n_classes} plant disease classes')

💾 Labels saved as data\models\labels\happy_plant_labels.pickle

🦕 Encoded labels:

	0: Background
	1: Bacterial_Spot
	2: Black_Measles
	3: Black_Rot
	4: Blight
	5: Citrus_Greening
	6: Healthy
	7: Leaf_Mold
	8: Leaf_Scorch
	9: Leaf_Spot
	10: Mosaic_Virus
	11: Powdery_Mildew
	12: Rust
	13: Scab
	14: Spider_Mites
	15: Target_Spot
	16: Yellow_Leaf_Curl_Virus

🪐 There are 17 plant disease classes


# Split data into train and validation set

In [13]:
X_train, X_valid, y_train, y_valid = train_test_split(
    X, 
    y_bin, 
    train_size=0.8, 
    random_state=42
)

In [14]:
assert X.shape[0] == y_bin.shape[0], 'X != y_bin'
assert X_train.shape[0] == y_train.shape[0], 'X_train != y_train'
assert X_valid.shape[0] == y_valid.shape[0], 'X_valid != y_valid'
assert X_test.shape[0] == y_test.shape[0], 'X_test != y_test'

# Build and train the model

In most convolutional networks, the higher up a layer is, the more specialized it is. The first few layers learn very simple and generic features that generalize to almost all types of images. As you go higher up, the features are increasingly more specific to the dataset on which the model was trained. The goal of fine-tuning is to adapt these specialized features to work with the new dataset, rather than overwrite the generic learning.

In [125]:
def compile_model(model, learning_rate):
    model.compile(
        optimizer=Adam(lr=learning_rate),
        loss=BinaryCrossentropy(from_logits=True),
        metrics=[
            CategoricalAccuracy(name='Accuracy'),
            Precision(name='Precision'), 
            Recall(name='Recall')
        ]
    )   
    return model

In [16]:
def train_model(
    model, 
    X_train, 
    y_train, 
    X_valid, 
    y_valid, 
    n_epochs=50,
    batch_size=32,
    apply_image_preprocessing=False,
    use_early_stopping=False,
    use_reduce_lr=False,
    use_checkpoint=False,
    checkpoint_path=''
):
    
    timestamp = datetime.strftime(datetime.now(), '%Y%m%d_%H%M%S')
    callbacks = []
    
    # Stop learning as soon as validation loss remains unchanged for 5 epochs
    if use_early_stopping:
        early_stopping = EarlyStopping(
            monitor ="val_loss",  
            mode ="min", 
            patience=5,  
            restore_best_weights=True,
            verbose=1
        )
        callbacks.append(early_stopping)

    # Checkpoint model weights 
    if use_checkpoint:
        if not checkpoint_path:
            checkpoint_path = os.path.join(MODELS_DIRECTORY, 'Model', 'model.hdf5')
        path, ext = os.path.splitext(checkpoint_path)
        checkpoint_path = f"{path}_{timestamp}{ext}"
        checkpoint = ModelCheckpoint(
            checkpoint_path,
            monitor='val_loss',
            mode='min',
            save_best_only=True,
            verbose=1
        )
        callbacks.append(checkpoint)
    
    # If no reducement of validation loss is present, learning rate is reduced
    if use_reduce_lr:
        reduce_lr = ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.2,
            patience=3,
            cooldown=5,
            verbose=1
        )
        callbacks.append(reduce_lr)

    if apply_image_preprocessing:
        image_gen = ImageDataGenerator(
            rotation_range=60,
            width_shift_range=0.2,
            height_shift_range=0.2,
            shear_range=0.2,
            zoom_range=0.2,
            brightness_range=[0.75, 1.25],
            horizontal_flip=True,
        )
        
        history = model.fit(
            image_gen.flow(X_train, y_train, batch_size=batch_size),
            validation_data=(X_valid, y_valid),
            epochs=n_epochs,
            callbacks=callbacks,
            verbose=1
        ).history
    
    else:
        history = model.fit(
            x=X_train,
            y=y_train,
            batch_size=batch_size,
            validation_data=(X_valid, y_valid),
            epochs=n_epochs,
            callbacks=callbacks,
            verbose=1
        ).history
    
    return history

# MobileNetV2

In [119]:
def trainable_layers(model) -> int:
    trainable = 0
    for layer in model.layers:
        if 'mobilenetv2' in layer.name:
            all_layers = len(layer.layers)
            trainable = sum([1 for mobile_layer in layer.layers if mobile_layer.trainable])
            return trainable - 1 if trainable < all_layers else trainable 
    return trainable

In [124]:
def mobile_net_v2(n_classes, learning_rate=1e-3, recent: str = "", limit_trainable: int = None):    
    if recent:
        print(f'[⚙️] Loading weights from {recent}')
        return load_model(recent)
    
    base_mobile_net_v2 = MobileNetV2(
        input_shape=INPUT_SHAPE,
        weights='imagenet',
        include_top=False
    )

    base_mobile_net_v2.trainable = True
    if isinstance(limit_trainable, int) and limit_trainable <= len(base_mobile_net_v2.layers):
        for layer in base_mobile_net_v2.layers[:-limit_trainable]:
            layer.trainable = False

    # Modify last layers
    input_layer = Input(shape=INPUT_SHAPE)
    hidden_layer = preprocess_input(input_layer)
    hidden_layer = base_mobile_net_v2(hidden_layer, training=False)
    hidden_layer = GlobalAveragePooling2D()(hidden_layer) 
    hidden_layer = Dense(1024, activation='relu')(hidden_layer)
    hidden_layer = Dense(1024, activation='relu')(hidden_layer)
    hidden_layer = Dense(512, activation='relu')(hidden_layer)
    hidden_layer = Dropout(rate=0.5)(hidden_layer)
    output_layer = Dense(n_classes, activation='softmax')(hidden_layer)

    mobile_net_v2_model = Model(inputs=input_layer, outputs=output_layer)

    return compile_model(mobile_net_v2_model, learning_rate)

# Fine-tuning

In [23]:
def generate_report(param_grid: dict) -> dict:
    return {parameter: {val: [] for val in values} for parameter, values in param_grid.items()}

In [60]:
def validation_results(report: dict):
    for parameter in report:
        results = sorted(report[parameter].items(), key=lambda item: item[1]['Accuracy'], reverse=True)
        for key, metrics in results:
            print(f'{parameter}: {key}')
            for metric, val in metrics.items():
                print(f' - {metric}: {val}')
            print('\n')

In [64]:
def parameter_validation(model_func, X_train, y_train, X_test, y_test, param_grid: dict, checkpoint_path: str = "", report: dict = {}) -> dict:
    if not report:
        report = generate_report(param_grid)
    
    for parameter in param_grid:
        for val in param_grid[parameter]:
            print(f'- {parameter}: {val}\n')
            model = model_func(n_classes, learning_rate=1e-4, **{parameter: val})
            train_model(
                model,
                X_train,
                y_train,
                X_valid,
                y_valid,
                n_epochs=50,
                batch_size=32,
                apply_image_preprocessing=True,
                use_reduce_lr=True,
                use_early_stopping=True,
                use_checkpoint=True,
                checkpoint_path=checkpoint_path
            )
            scores = model.evaluate(X_test, y_test, verbose=2)
            report[parameter][val] = dict(zip(['Accuracy', 'Precision', 'Recall'], scores[1:]))
            print('\n')

## MobileNetV2 fine-tuning

In [65]:
mobile_net_v2_checkpoint_path = os.path.join(MODELS_DIRECTORY, 'MobileNetV2', 'mobile_net_v2.hdf5')
y_test_bin = encode_labels(y_test, label_binarizer)

mobile_net_v2_grid = {'limit_trainable': [25, 50, 100, 125, "All"]}
mobile_net_v2_results = generate_report(mobile_net_v2_grid)
parameter_validation(
    model_func=mobile_net_v2,
    X_train=X_train, 
    y_train=y_train, 
    X_test=X_test, 
    y_test=y_test_bin, 
    param_grid=mobile_net_v2_grid, 
    checkpoint_path=mobile_net_v2_checkpoint_path,
    report=mobile_net_v2_results
)

- limit_trainable: 25

Epoch 1/50
278/278 [==============================] - 123s 396ms/step - loss: 0.2036 - accuracy: 0.3910 - precision: 0.6265 - Recall: 0.2629 - val_loss: 0.0603 - val_accuracy: 0.8047 - val_precision: 0.8579 - val_Recall: 0.7627

Epoch 00001: val_loss improved from inf to 0.06032, saving model to data\models\MobileNetV2\mobile_net_v2_20210516_145033.hdf5
Epoch 2/50
278/278 [==============================] - 106s 382ms/step - loss: 0.0702 - accuracy: 0.7968 - precision: 0.8572 - Recall: 0.7509 - val_loss: 0.0381 - val_accuracy: 0.8787 - val_precision: 0.8957 - val_Recall: 0.8642

Epoch 00002: val_loss improved from 0.06032 to 0.03809, saving model to data\models\MobileNetV2\mobile_net_v2_20210516_145033.hdf5
Epoch 3/50
278/278 [==============================] - 108s 389ms/step - loss: 0.0452 - accuracy: 0.8665 - precision: 0.8951 - Recall: 0.8467 - val_loss: 0.0359 - val_accuracy: 0.8913 - val_precision: 0.9038 - val_Recall: 0.8859

Epoch 00003: val_loss improved f

## MobileNetV2 CV Results

In [66]:
validation_results(mobile_net_v2_results)

limit_trainable: 100
 - Accuracy: 0.9771959185600281
 - Precision: 0.9771766662597656
 - Recall: 0.9763513803482056


limit_trainable: All
 - Accuracy: 0.9755067825317383
 - Precision: 0.9754860401153564
 - Recall: 0.974662184715271


limit_trainable: 125
 - Accuracy: 0.974662184715271
 - Precision: 0.9746407270431519
 - Recall: 0.9738175868988037


limit_trainable: 25
 - Accuracy: 0.9679054021835327
 - Precision: 0.9678782820701599
 - Recall: 0.9670608043670654


limit_trainable: 50
 - Accuracy: 0.9619932174682617
 - Precision: 0.9619932174682617
 - Recall: 0.9619932174682617




# Predict

In [1]:
def predict_disease(model, encoded_labels, x_test: str or np.ndarray, y_true=None, plot=False):
    if os.path.isfile(x_test):
        image = load_img(x_test)
        image_array = img_to_array(image)
    else:
        image = array_to_img(x_test)
        image_array = x_test
    
    y_pred = np.argmax(model.predict(np.expand_dims(image_array, axis=0)), axis=-1)[0]
        
    print(f'Prediction: {encoded_labels[y_pred]}')
    if y_true:
        print(f'Actual Disease: {y_true}')
    
    if plot:
        plt.imshow(image)
        plt.show()

# Reuse model

In [129]:
def get_latest_model(root_dir: str) -> str:
    return max(glob.iglob(os.path.join(root_dir, '*.hdf5')), key=os.path.getctime)

In [108]:
y_test_bin = encode_labels(y_test, label_binarizer)

In [25]:
new_mobile_net_v2 = mobile_net_v2(n_classes, recent=get_latest_model(r'data/models/MobileNetV2/'), limit_trainable=50)
mobile_net_v2_scores = new_mobile_net_v2.evaluate(X_test, y_test_bin, verbose=2)

[⚙️] Loading weights from data/models/MobileNetV2\mobile_net_v2_20210515_211513.hdf5
37/37 - 4s - loss: 0.0102 - accuracy: 0.9755 - precision: 0.9755 - Recall: 0.9755
